In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection)
from sklearn.cluster import KMeans, DBSCAN
import pandas as pd
# 数据库访问
import MySQLdb, datetime

## 读取数据（user_dim.txt）

In [2]:
X = np.loadtxt('/Users/dujiawei/git/UserAnalysis/result/user_dim.txt')
# Open database connection
conn = MySQLdb.connect(host='localhost',user='root',passwd='',db='qyw', charset='utf8')

## t-SNE算法降维+DBSCAN聚类（效果最好）

In [3]:
# ###############################################################################
# not good
# Compute Kmeans
# # Visualize the results on PCA-reduced data
# tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
# reduced_data = tsne.fit_transform(X)
# kmeans = KMeans(init='k-means++', n_clusters=100, n_init=1)
# cls = kmeans.fit(reduced_data)
# print cls.cluster_centers_
# print cls.labels_
# print cls.inertia_

##############################################################################
# Visualize the results on PCA-reduced data
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
reduced_data = tsne.fit_transform(X)
plt.scatter(reduced_data[:, 0], reduced_data[:, 1])
plt.savefig('result/user_dr_tsne.png')
# plt.show()
# Compute DBSCAN
# db = DBSCAN(eps=12, min_samples=10).fit(reduced_data) # 低维数据
db = DBSCAN(eps=4, min_samples=10).fit(X) # 高维数据
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
print labels
print n_clusters_
print len(labels[labels>=0])

plt.scatter(reduced_data[:, 0], reduced_data[:, 1], 20, labels) # 20为散点的直径
plt.savefig('result/user_cluster_dbscan.png')
plt.show()
plt.cla()
plt.clf()
plt.close()

## 展示用户聚类结果（散点图和excel表格）
USERS = np.loadtxt('/Users/dujiawei/git/UserAnalysis/result/user.txt')
USERS_CLS =  np.hstack((USERS.reshape(USERS.size, 1),labels.reshape(labels.size,1)))
print USERS_CLS.shape
np.savetxt('result/user_cls.txt', USERS_CLS, fmt='%d')
USERS_CLS_DF = pd.DataFrame(USERS_CLS, columns=['USER_ID', 'CLS_ID'])
print USERS_CLS_DF.shape
USERS_CLS_DF.to_sql('qyw_7th_user_clusters', conn, flavor='mysql', if_exists='replace', index=False)
USERS_CLS_CNTCASE = pd.read_sql('''
    SELECT t1.*,
       t2.CLS_ID
FROM
  (SELECT USER_ID,
          COUNT(DISTINCT CASE_ID) AS CNT_CASE
   FROM qyw_7th_yy_succ_all_selected
   GROUP BY USER_ID) AS t1
INNER JOIN
  (SELECT *
   FROM qyw_7th_user_clusters) AS t2 ON t1.USER_ID = t2.USER_ID
ORDER BY t1.CNT_CASE DESC;
''', con=conn)
c = ['y','r']
colors = [c[int(i)] for i in np.array(USERS_CLS_CNTCASE['CLS_ID'])]
plt.scatter(USERS_CLS_CNTCASE['CNT_CASE'], USERS_CLS_CNTCASE['USER_ID'], 20, colors) # 20为散点的直径
plt.savefig('result/user_cls_cntcase.png')
plt.show()
plt.cla()
plt.clf()
plt.close()
USERS_CLS_CNTCASE.to_excel('result/user_cls_cntcase.xls')
conn.close()

[0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1
 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 1 1]
2
120
(120, 2)

/Users/dujiawei/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):
/Users/dujiawei/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/pandas/core/generic.py:1003: FutureWarning: The 'mysql' flavor with DBAPI connection is deprecated and will be removed in future versions. MySQL will be further supported with SQLAlchemy connectables.
  dtype=dtype)



(120, 2)


## 其他降维算法

In [ ]:
#----------------------------------------------------------------------
# Random 2D projection using a random unitary matrix
print("Computing random projection")
rp = random_projection.SparseRandomProjection(n_components=2, random_state=42)
X_projected = rp.fit_transform(X)
plt.scatter(X_projected[:, 0], X_projected[:, 1])
plt.show()

In [ ]:
#----------------------------------------------------------------------
# Projection on to the first 2 principal components
print("Computing PCA projection")
X_pca = decomposition.TruncatedSVD(n_components=2).fit_transform(X)
plt.scatter(X_pca[:, 0], X_pca[:, 1])
plt.show()

In [ ]:
#----------------------------------------------------------------------
# Isomap projection of the digits dataset
print("Computing Isomap embedding")
n_neighbors = 3
X_iso = manifold.Isomap(n_neighbors, n_components=2).fit_transform(X)
print("Done.")
plt.scatter(X_iso[:, 0], X_iso[:, 1])
plt.show()

In [ ]:
#----------------------------------------------------------------------
# Locally linear embedding of the digits dataset
print("Computing LLE embedding")
n_neighbors = 3
clf = manifold.LocallyLinearEmbedding(n_neighbors, n_components=2,
                                      method='standard')
X_lle = clf.fit_transform(X)
print("Done. Reconstruction error: %g" % clf.reconstruction_error_)
plt.scatter(X_lle[:, 0], X_lle[:, 1])
plt.show()

In [ ]:
## good
#----------------------------------------------------------------------
# Modified Locally linear embedding of the digits dataset
print("Computing modified LLE embedding")
n_neighbors = 3
clf = manifold.LocallyLinearEmbedding(n_neighbors, n_components=2,
                                      method='modified')
X_mlle = clf.fit_transform(X)
print("Done. Reconstruction error: %g" % clf.reconstruction_error_)
plt.scatter(X_mlle[:, 0], X_mlle[:, 1])
plt.show()

In [ ]:
## good
#----------------------------------------------------------------------
# HLLE embedding of the digits dataset
print("Computing Hessian LLE embedding")
n_neighbors = 6
clf = manifold.LocallyLinearEmbedding(n_neighbors, n_components=2,
                                      method='hessian')
X_hlle = clf.fit_transform(X)
print("Done. Reconstruction error: %g" % clf.reconstruction_error_)
plt.scatter(X_hlle[:, 0], X_hlle[:, 1])
plt.show()

In [ ]:
## good
#----------------------------------------------------------------------
# LTSA embedding of the digits dataset
print("Computing LTSA embedding")
n_neighbors = 100
clf = manifold.LocallyLinearEmbedding(n_neighbors, n_components=2,
                                      method='ltsa')
X_ltsa = clf.fit_transform(X)
print("Done. Reconstruction error: %g" % clf.reconstruction_error_)
plt.scatter(X_ltsa[:, 0], X_ltsa[:, 1])
plt.show()

In [ ]:
## good
#----------------------------------------------------------------------
# MDS  embedding of the digits dataset
print("Computing MDS embedding")
clf = manifold.MDS(n_components=2, n_init=1, max_iter=100)
X_mds = clf.fit_transform(X)
print("Done. Stress: %f" % clf.stress_)
plt.scatter(X_mds[:, 0], X_mds[:, 1])
plt.show()

In [ ]:
## good
#----------------------------------------------------------------------
# Random Trees embedding of the digits dataset
print("Computing Totally Random Trees embedding")
hasher = ensemble.RandomTreesEmbedding(n_estimators=200, random_state=0,
                                       max_depth=5)
X_transformed = hasher.fit_transform(X)
pca = decomposition.TruncatedSVD(n_components=2)
X_reduced = pca.fit_transform(X_transformed)
plt.scatter(X_reduced[:, 0], X_reduced[:, 1])
plt.show()

In [ ]:
#----------------------------------------------------------------------
# Spectral embedding of the digits dataset
print("Computing Spectral embedding")
embedder = manifold.SpectralEmbedding(n_components=2, random_state=0,
                                      eigen_solver="arpack")
X_se = embedder.fit_transform(X)
plt.scatter(X_se[:, 0], X_se[:, 1])
plt.show()

In [ ]:
## best
#----------------------------------------------------------------------
# t-SNE embedding of the digits dataset
print("Computing t-SNE embedding")
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
X_tsne = tsne.fit_transform(X)
plt.scatter(X_tsne[:, 0], X_tsne[:, 1])
plt.show()